# EOEPCA Data Access Validation and Usage Notebook

## Setup

In [2]:
import os
import requests
import json
import matplotlib.pyplot as plt
from PIL import Image

from owslib.csw import CatalogueServiceWeb
from owslib.wms import WebMapService
from owslib.wcs import WebCoverageService
import lxml.etree
from tifffile import imread
from io import BytesIO, StringIO

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [3]:
load_eoepca_state()

In [4]:
platform_domain = os.environ.get("INGRESS_HOST")
data_access_domain = f'{os.environ.get("HTTP_SCHEME")}://eoapi.{platform_domain}'
stac_api_domain = f'{os.environ.get("HTTP_SCHEME")}://resource-catalogue.{platform_domain}/stac'
csw_domain = f'{os.environ.get("HTTP_SCHEME")}://resource-catalogue.{platform_domain}/csw'
# stac_api_domain = 'https://eoapi.notebook-test.develop.eoepca.org/stac'

print(f"Data Access API: {data_access_domain}")
print(f"STAC API: {stac_api_domain}")

Data Access API: https://eoapi.notebook-test.develop.eoepca.org
STAC API: https://resource-catalogue.notebook-test.develop.eoepca.org/stac


## Validate Data Access API Endpoints

In [5]:
endpoints = [
    ("STAC API Landing Page", f"{data_access_domain}/stac/"),
    ("STAC Swagger UI", f"{data_access_domain}/stac/api.html"),
    ("Raster API Swagger UI", f"{data_access_domain}/raster/api.html"),
    ("Vector API Swagger UI", f"{data_access_domain}/vector/api.html")
]

for name, url in endpoints:
    response = requests.get(url)
    print(f"{name} ({url}): {response.status_code}")

STAC API Landing Page (https://eoapi.notebook-test.develop.eoepca.org/stac/): 200
STAC Swagger UI (https://eoapi.notebook-test.develop.eoepca.org/stac/api.html): 200
Raster API Swagger UI (https://eoapi.notebook-test.develop.eoepca.org/raster/api.html): 200
Vector API Swagger UI (https://eoapi.notebook-test.develop.eoepca.org/vector/api.html): 200


In [6]:
csw = CatalogueServiceWeb(csw_domain,timeout=30)
scene_id="S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE"
csw.getrecordbyid(id=[scene_id])
links = csw.records[scene_id].references

for link in links:
    scheme = link['scheme']
    if scheme and 'WMS' in scheme:
        wms_endpoint=link['url']
        print(link['url'])

https://data-access-v1x.develop.eoepca.org/ows?rel=http%3A%2F%2Fwww.opengis.net%2Fdef%2FserviceType%2Fogc%2Fwms&service=WMS&version=1.3.0&request=GetCapabilities&cql=identifier%3D%22S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE%22


## Data Visualisation

In [7]:
wms = WebMapService(wms_endpoint, version='1.3.0')
print(list(wms.contents))
print(wms[scene_id].title)


['S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE', 'S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE__outlines', 'S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE__outlined', 'S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE__FALSE_COLOR', 'S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE__NDVI', 'S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE__TRUE_COLOR']
S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE


/home/eouser/code/deployment-guide/venv/lib/python3.10/site-packages/owslib/map/wms130.py:121: UserWarning: Content metadata for layer "S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE" already exists. Using child layer
  warnings.warn('Content metadata for layer "%s" already exists. Using child layer' % cm.id)


In [8]:
import os
import folium

print(folium.__version__)

import folium.plugins.timestamped_wmstilelayer

0.19.5


In [10]:
centre_lat=wms[scene_id].boundingBoxWGS84[1]+(wms[scene_id].boundingBoxWGS84[3]-wms[scene_id].boundingBoxWGS84[1])/2
centre_long=wms[scene_id].boundingBoxWGS84[0]+(wms[scene_id].boundingBoxWGS84[2]-wms[scene_id].boundingBoxWGS84[0])/2
m = folium.Map(location=[centre_lat, centre_long], zoom_start=7, tiles=None)

folium.raster_layers.WmsTileLayer(
    url="https://a.tiles.maps.eox.at",
    layers='terrain-light_3857',
    name='terrain-light',
    fmt='image/jpeg',
).add_to(m)


folium.raster_layers.WmsTileLayer(
    url=wms_endpoint.partition("?")[0],
    layers=f"{scene_id}__TRUE_COLOR",
    name=f"{scene_id}__TRUE_COLOR",
    fmt='image/png',
    transparent=True,
    overlay=True,
    control=True,
).add_to(m)

folium.raster_layers.WmsTileLayer(
    url=wms_endpoint.partition("?")[0],
    layers=f"{scene_id}__outlines",
    name=f"{scene_id}__outlines",
    fmt='image/png',
    transparent=True,
    overlay=True,
    control=True,
).add_to(m)

folium.LayerControl().add_to(m)

m

## Whole Timespan

In [11]:
m = folium.Map(location=[centre_lat, centre_long], zoom_start=6, tiles=None)

folium.raster_layers.WmsTileLayer(
    url="https://a.tiles.maps.eox.at",
    layers='terrain-light_3857',
    name='terrain-light',
    fmt='image/jpeg',
).add_to(m)

folium.raster_layers.WmsTileLayer(
    url=wms_endpoint.partition("?")[0],
    layers='S2L2A__outlines',
    name='Outlines',
    fmt='image/png',
    transparent=True,
    overlay=True,
    control=True,
    time="2019-09-10T00:00:00Z/2019-09-11T00:00:00Z",
).add_to(m)


folium.LayerControl().add_to(m)

m

## Data Download

In [12]:
tree = None
for link in links:
    scheme = link['scheme']
    if scheme and 'WCS' in scheme:
        print(link['url'])
        wcs_endpoint=link['url'].split('?')[0]
        wcs_id=link['url'].split('eoid=')[1]
        tree = lxml.etree.fromstring(requests.get(link['url']).content)
        break

coverage_ids = tree.xpath('wcs:CoverageDescriptions/wcs:CoverageDescription/@gml:id', namespaces=tree.nsmap)
coverage_ids

https://data-access-v1x.develop.eoepca.org/ows?rel=http%3A%2F%2Fwww.opengis.net%2Fdef%2FserviceType%2Fogc%2Fwcs&service=WCS&version=2.0.1&request=DescribeEOCoverageSet&eoid=S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE


['S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE_B01',
 'S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE_B02',
 'S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE_B03',
 'S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE_B04',
 'S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE_B05',
 'S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE_B06',
 'S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE_B07',
 'S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE_B08',
 'S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE_B09']

In [13]:
getcoverage_request = wcs_endpoint + '?service=WCS&version=2.0.1&request=GetCoverage&coverageid=' + coverage_ids[1] + '&scaleSize=x(100),y(100)&format=image/tiff'
print(getcoverage_request)
response = requests.get(getcoverage_request)
response.raise_for_status()

content = response.content
img1 = plt.imshow(imread(BytesIO(content)),extent=[23.4,24.5,37.8,38.8],aspect='auto')
plt.show()

https://data-access-v1x.develop.eoepca.org/ows?service=WCS&version=2.0.1&request=GetCoverage&coverageid=S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE_B02&scaleSize=x(100),y(100)&format=image/tiff


KeyboardInterrupt: 

## STAC Collections and Items Inspection

In [18]:
collections_response = requests.get(f"{data_access_domain}/stac/collections")
collections = collections_response.json()
print("Collections:")
for collection in collections['collections']:
    print(f" - {collection['id']}")

Collections:
 - noaa-emergency-response
 - sentinel-2-iceland


## Load and Inspect Sample STAC Collection and Items

TODO: Why doesn't resource registration register it in?

In [19]:
collections_url = "https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/scripts/data-access/collections/sentinel-2-iceland/collections.json"
items_url = "https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/scripts/data-access/collections/sentinel-2-iceland/items.json"

collections_json = requests.get(collections_url).json()
items_json = requests.get(items_url).json()

## Ingestion of Sample Collection and Items

In [20]:
collections_endpoint = f"{data_access_domain}/stac/collections"
ingested_collection = requests.post(collections_endpoint, json=collections_json)
print(f"Collection ingestion status code: {ingested_collection.status_code}")
print(ingested_collection.json())

items_endpoint = f"{data_access_domain}/stac/collections/{collections_json['id']}/items"

successful_items = []
failed_items = []

# only take first 5 items for testing
items_json = items_json[:5]

for item in items_json:
    response = requests.post(items_endpoint, json=item)
    if response.status_code == 200:
        successful_items.append(item['id'])
    else:
        failed_items.append((item['id'], response.text))

print(f"Successfully ingested items: {successful_items}")
if failed_items:
    print(f"Failed items: {failed_items}")

Collection ingestion status code: 409
{'code': 'ConflictError', 'description': ''}
Successfully ingested items: []
Failed items: [('S2A_26WPS_20231109_0_L2A', '{"code":"ConflictError","description":""}'), ('S2B_27VWL_20231109_0_L2A', '{"code":"ConflictError","description":""}'), ('S2B_26WPS_20231108_0_L2A', '{"code":"ConflictError","description":""}'), ('S2B_26WNT_20231108_0_L2A', '{"code":"ConflictError","description":""}'), ('S2B_26WPT_20231108_0_L2A', '{"code":"ConflictError","description":""}')]


## STAC Items Search and Visualization (Post-Ingestion)

In [21]:
collection_check = requests.get(f"{collections_endpoint}/{collections_json['id']}")
print(f"Collection retrieval status: {collection_check.status_code}")

items_check = requests.get(items_endpoint)
print(f"Items retrieval status: {items_check.status_code}")

Collection retrieval status: 200
Items retrieval status: 200


In [22]:
search_url = f"{data_access_domain}/stac/search"
search_payload = {
    "bbox": [-25, 63, -24, 65],
    "datetime": "2023-01-01T00:00:00Z/2024-01-01T00:00:00Z",
    "limit": 5
}

response = requests.post(search_url, json=search_payload)
search_results = response.json()
print(json.dumps(search_results, indent=2)[:100])

{
  "type": "FeatureCollection",
  "links": [
    {
      "rel": "root",
      "type": "application/


In [23]:
if search_results['features']:
    stac_item = search_results['features'][0]
    visual_asset_url = stac_item['assets']['visual']['href']
    print(f"Visual asset URL: {visual_asset_url}")

    # image_response = requests.get(visual_asset_url)
    # img = Image.open(BytesIO(image_response.content))

    # plt.figure(figsize=(8, 8))
    # plt.imshow(img)
    # plt.axis('off')
    # plt.title(f"Raster Visualization: {stac_item['id']}")
    # plt.show()
else:
    print("No STAC items found.")

Visual asset URL: https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/26/W/PS/2023/11/S2A_26WPS_20231109_0_L2A/TCI.tif


## Raster and Vector API Availability Checks

In [24]:
raster_health_url = f"{data_access_domain}/raster/healthz"
vector_health_url = f"{data_access_domain}/vector/healthz"

raster_health = requests.get(raster_health_url)
vector_health = requests.get(vector_health_url)

print(f"Raster API Health Check: {raster_health.status_code} - {raster_health.text}")
print(f"Vector API Health Check: {vector_health.status_code} - {vector_health.text}")


Raster API Health Check: 200 - {"database_online":true}
Vector API Health Check: 200 - {"ping":"pong!"}


## STAC Manager UI Check

In [25]:
stac_manager_ui_url = data_access_domain
response = requests.get(stac_manager_ui_url)
print(f"STAC Manager UI status code: {response.status_code}")
response.url

STAC Manager UI status code: 200


'https://eoapi.notebook-test.develop.eoepca.org/'